In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install
!pip install Pyphen
!pip install langdetect
!pip install pyspellchecker
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-18 16:41:58.817594: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
print("Loading libraries ...")
import pandas as pd
import spacy
import pyphen
from datetime import datetime
import string
import re
import numpy as np
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk import Tree
from textblob import TextBlob
from langdetect import detect
from langdetect import detect_langs
from spellchecker import SpellChecker
import collections as ct
from datetime import datetime
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
print('loaded')

Loading libraries ...


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


loaded


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
nlp = spacy.load("en_core_web_lg")
spell = SpellChecker()
stopwords = nlp.Defaults.stop_words
dic = pyphen.Pyphen(lang='en')
sentiment = SentimentIntensityAnalyzer()
p_stemmer = PorterStemmer()
tfidf_vectorizer=TfidfVectorizer(use_idf=True, stop_words=stopwords) 
print('Initialization done')

Initialization done


In [18]:
df = pd.read_excel('/content/drive/MyDrive/output_user_agent_no_names.xlsx', sheet_name=0)
print("Succesfully read excel")

Succesfully read excel


In [11]:
class Scoring:
    def __init__(self, ticket='', unclean_ticket=''):
        self.unclean_ticket = unclean_ticket
        self.ticket = ticket
        if not isinstance(self.ticket, str):
            self.ticket = ''
        # self.doc = nlp(self.ticket)
        self.alph = re.sub(r'[^a-zA-Z ]', '', self.ticket)
        self.words = list(filter(None, self.alph.split(' ')))
        # self.entities_sw = [i for i in self.doc.ents if i.label_ not in ['ORG', 'WORK_OF_ART', 'PRODUCT']]
        self.stopwords = stopwords

    def get_sentences(self):
        sentences = [sent.text.strip() for sent in self.doc.sents]
        for i, sent in enumerate(sentences):
            sent = re.sub(r'[^a-zA-Z]', '', sent)
            if sent == '':
                sentences[i] = ''
        sentences = list(filter(None, sentences))
        if len(sentences) == 0:
            return None, 1
        return sentences, len(sentences)

    def get_words(self):
        return len(self.words)

    def words_per_sentences(self):
        if self.get_sentences()[1] > 0:
            return float("{:.2f}".format(self.get_words() / self.get_sentences()[1]))
        else:
            return self.get_words()

    def no_stopwords(self):
        sw_count = 0
        for word in self.words:
            if word.lower() in self.stopwords:
                sw_count += 1
        # sw_count += len(self.entities_sw)
        no_sw = self.get_words() - sw_count
        return sw_count, self.get_words() - sw_count, float("{:.2f}".format(no_sw / self.get_sentences()[1]))

    def get_nouns(self):
        n_count = 0
        for noun in self.doc.noun_chunks:
            if str(noun).lower() not in self.stopwords:
                n_count += 1
        return n_count

    def get_pronouns(self):
        pn_count = 0
        for pn in self.doc:
            if pn.tag_[0] == 'P':
                pn_count += 1
        return pn_count

    def get_readablity(self):
        suffix = ["ing", "ed", "es", "ous", "tion", "nce", "ness"]
        complex_words = []
        new_ticket = self.ticket.translate(str.maketrans('', '', string.punctuation))
        w = float(self.get_words())
        s = float(self.get_sentences()[1])
        # for ent in self.entities_sw:
        #     new_ticket = new_ticket.replace(str(ent), '')
        new_ticket = new_ticket.replace('-', '')
        syllables_count = 0
        for wrd in new_ticket.split(' '):
            ws = dic.inserted(wrd).count('-')
            syllables_count += ws + 1
            for suf in suffix:
                if wrd.endswith(suf):
                    ws -= 1
            if ws >= 2:
                complex_words.append(wrd)
        # words_count_nt = self.get_words() - len(self.entities_sw)
        # Flesch
        syllables_count = float(syllables_count)
        if s == 0 or w == 0:
          score1 = 0
          score2 = 0
          score3 = 0
        else:
          score1 = 206.835 - 1.015 * w / s - 84.6 * syllables_count / w
          # Coleman–Liau
          letters = float(len(new_ticket) - new_ticket.count(' '))
          score2 = 0.0588 * letters / w * 100 - 0.296 * s / w * 100 - 15.8
          # Gunning fog
          score3 = 0.4 * (w / s + 100 * float(len(complex_words)) / w)
        return float("{:.2f}".format(score1)), float("{:.2f}".format(score2)), float("{:.2f}".format(score3))

    def get_verbs(self):
        v_count = 0
        for v in self.doc:
            if v.tag_[0] == 'V':
                v_count += 1
        return v_count

    @staticmethod
    def get_links(ticket_original):
        links = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+', ticket_original)
        links = filter(lambda x: x[:5] != 'image', links)
        flinks = list(links)
        return flinks, len(flinks)

    @staticmethod
    def get_attachments(ticket_original):
        c = 0
        cc = 0
        if '[' in ticket_original and ']' in ticket_original:
            new_s = ticket_original
            while '[' in new_s:
                ia = new_s.find("[")
                text = new_s[new_s.find("[") + 1:new_s.find("]", ia)]
                if text != '[' and text != ']' and len(text) > 0:
                    if text == 'code':
                        cc += 1
                    else:
                        c += 1
                new_s = new_s[:ia] + new_s[ia + 1:]
        return int(cc / 2) + c

    @staticmethod
    def get_interacts(ticket_original):
        matches = re.findall(r'\d{4}-\d{2}-\d{2} \d{2}\d{2}\d{2}', ticket_original)
        return len(matches)

    def preprocess_lem(self):
        alph_ticket = self.alph
        lemdoc = nlp(alph_ticket)
        lemticket = ' '.join([token.lemma_ for token in lemdoc])
        return lemticket

    def get_sentiment(self):
        ticket = Scoring(self.ticket).preprocess_lem()
        sentiment_score = TextBlob(ticket).sentiment.polarity
        if sentiment_score < 0:
            sentiment = 'Negative'
        elif sentiment_score == 0:
            sentiment = 'Neutral'
        else:
            sentiment = 'Positive'

        return sentiment_score, sentiment

    def get_refrences(self):
        ticket_low = self.ticket.lower()
        return ticket_low.count('called') + ticket_low.count('call') + ticket_low.count('sent') + ticket_low.count(
            'send') + ticket_low.count('messaged')

    @staticmethod
    def get_datediff(date1, date2):
        date_format = "%Y-%m-%d"
        a = datetime.strptime(date1, date_format)
        b = datetime.strptime(date2, date_format)
        c = datetime.now()
        delta = b - a
        delta2 = c - a
        return delta.days, delta2.days

    def get_language_confidence(self):
        ticket = Scoring(self.ticket).preprocess_lem()
        confidence = str(detect_langs(ticket)[0]).partition(':')[2]
        confidence = float(confidence)

        return confidence

    def get_special_characters(self):
        special_chars = ['#', '+', '~', '/', '[', ']', '<', '>', '{', '}']
        return sum(v for k, v in ct.Counter(self.unclean_ticket).items() if k in special_chars)

    def get_bulleted_list(self):
        bullet_style = ['#', '*', '++', '-']
        count_bullet_style = 0
        for style in bullet_style:
            count = self.unclean_ticket.count(style)
            if count < 7 and count >= 3:
                count_bullet_style += 1

        check = False
        if count_bullet_style > 0:
            check = True

        return check, count_bullet_style
      
    def get_spelling_mistakes(self):
        ticket = re.findall("[a-zA-Z,.]+", self.ticket)
        words = re.findall("[a-zA-Z,]+", self.ticket)
        misspelled = spell.unknown(ticket)

        return len(misspelled) / len(words)

    def get_verb_noun_combinations(self):
        nosw_ticket = ' '.join([w for w in self.ticket.split(' ') if w not in self.stopwords])
        pos_doc1 = nlp(nosw_ticket)
        pos_doc2 = self.doc
        pos_list1 = ""
        pos_list2 = ""
        for token in pos_doc1:
            pos_list1 += ' ' + (token.pos_)
        for token in pos_doc2:
            pos_list2 += ' ' + (token.pos_)
        return pos_list1.count('VERB NOUN'), pos_list2.count('VERB NOUN')

    def to_nltk_tree(self, node):
        if node.n_lefts + node.n_rights > 0:
            return Tree(node.orth_, [self.to_nltk_tree(child) for child in node.children])
        else:
            return node.orth_

    def get_max_depth(self):
        nlp_text = nlp(self.ticket)
        max_depth = 0
        for sentence in nlp_text.sents:
            tree = self.to_nltk_tree(sentence.root)
            if not isinstance(tree, str):
                    max_depth = max(max_depth, tree.height())
        return max_depth

    def get_specific_words_count(self):
        text = [word for word in self.words if len(word) > 3]
        stemmed_tokens = [p_stemmer.stem(i) for i in text]
        en_filtered = [w for w in stemmed_tokens if not w in self.stopwords]
        stemmed_tokens = [p_stemmer.stem(i) for i in en_filtered]
        return len(stemmed_tokens)

    def get_unknown_words_count(self):
        ticket = self.ticket
        with open('/content/drive/MyDrive/unknown_words.txt') as f:
            lines = f.readlines()
        unknown_words = [x.replace('\n', '') for x in lines]
        new_unknown_words = []
        for word in unknown_words:
            new_unknown_words.append(word.split())
        flattened  = [val for sublist in new_unknown_words for val in sublist]
        final_unknown_words = [x.replace(',', '') for x in flattened]
        count_unknown_words = 0
        for unknown_word in final_unknown_words:
            count_unknown_words += ticket.count(' ' + unknown_word + ' ')
        return count_unknown_words

    def get_inquery_intensity(self): 
        return self.ticket.count('?')

In [7]:
def remove_lines_with_date_start(text):
    try:
        splitted_text = text.split("\n")
    except:
        splitted_text = []
    new_text = ""
    for i, sp_text in enumerate(splitted_text):
        if not re.match("[0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}[0-9]{2}[0-9]{2}", sp_text[0:19]):
            new_text += sp_text + "\n"
        else:
            if not re.match("[0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}[0-9]{2}[0-9]{2}", splitted_text[i - 1]):
                new_text += "______________________________" + "\n"
            else:
                new_text += "\n"
    return new_text.rstrip()


def remove_lines_start(text, start):
    greetings = ["good day", "hi", "hello", "dear", "many thanks", "thank you"]
    try:
        splitted_text = text.split("\n")
    except:
        splitted_text = []
    new_text = ""
    for sp_text in splitted_text:
        cond = True
        if not sp_text[0:len(start)] == start:
            clean_sp_text = ''.join([d for d in sp_text if not d.isdigit()])
            clean_sp_text = clean_sp_text.replace('.', '')
            clean_sp_text = clean_sp_text.replace('-', '')
            clean_sp_text_list = list(filter(None, clean_sp_text.strip().split(" ")))
            # print(clean_sp_text_list)
            if len(clean_sp_text_list) <= 5:
                for greeting in greetings:
                    if clean_sp_text.lower().strip().startswith(greeting):
                        cond = False
                if len(clean_sp_text_list) <= 1:
                    cond = False

            if sp_text == "______________________________":
                cond = True
            if cond:
                new_text += sp_text + "\n"

    return new_text.rstrip()


def preprocess(ticket):
    ticket = '\n' + str(ticket)

    # remove names after @
    for i, w in enumerate(ticket):
        if w == '@':
            y = ticket.find(' ', i)
            z = ticket.find(' ', y+1)
            ticket = ticket.replace(ticket[i:z], '')

    index = [m.start() for m in re.finditer(re.escape('code]'), ticket)]
    if len(index) > 0:
        for i, ind in enumerate(index):
            try:
                index = [m.start() for m in re.finditer(re.escape('code]'), ticket)]
                start_index = index[0] - 1
                end_index = index[1] + 5
                ticket = ticket[:start_index] + ticket[end_index + 1:]
                index = [m.start() for m in re.finditer(re.escape('code]'), ticket)]
                if len(index) < 2:
                    break
            except:
                continue

    extras = ['[', ']', 'cid', '+', '(', ')', ':', "xD",
              '<', '>', '#', '~', '/', '{', '}', "Issue description", "_x000D_", ';', ',', "Pvt  Limited", "Short Description"
              , "My action taken", "Issues", "Action taken", "Directory Services Administrator", "PM", "AM", "nan"]

    for extra in extras:
        ticket = ticket.replace(extra, '')

    text = "The information contained in this communication"
    if text in ticket:
        extra_text_index = ticket.find("The information contained in this communication")
        extra_lines = ticket[extra_text_index:].split('\n')[:9]
        for line in extra_lines:
            ticket = ticket.replace(line, '')

    ticket = re.sub('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+', '', ticket)

    ticket = '\n'.join(line.strip() for line in ticket.split('\n') if line != '' and "khayaban" not in line.lower())
    line_beginnings_remove = ["Additional Location Information", "Affected Device", "received from", "Email",
                              "Tel", "Subject", "From", "Sent", "To", "reply from", "Issue(s)"
                              , "Who called", "Ext Number", "Locations", "Device", "Call time", "Computer", "Client Type", "Version VPN", "PC Name", "Keyword",
                              "Sent", "From"]


    for beginning in line_beginnings_remove:
        ticket = remove_lines_start(ticket, beginning)

    ticket_with_time = ticket
    ticket = remove_lines_with_date_start(ticket)
    doc = nlp(ticket)
    names = [i for i in doc.ents if i.label_.lower() in ["person", "gpe", "location"]]
    for name in names:
        ticket = ticket.replace(str(name), '')
    ticket = '\n '.join(line.strip() for i, line in enumerate(ticket.split('\n')) if line != '' and (1 < i < len(ticket.split('\n')) - 1 or line != "______________________________"))
    return ticket.strip(), ticket_with_time.strip()

In [ ]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'number', 'opened_by',
       'short_description', 'priority', 'assignment_group', 'assigned_to',
       'u_main_category_reporting', 'u_subcategory_1_reporting',
       'u_subcategory_2_reporting', 'u_subcategory_3_reporting',
       'u_resolver_group', 'u_resolver', 'u_first_assignment_group',
       'u_country', 'sys_updated_on', 'sys_updated_by', 'sys_mod_count',
       'u_vip', 'u_type', 'u_region', 'contact_type', 'location', 'u_resolved',
       'closed_at', 'u_assignment_group_history', 'work_notes', 'comments',
       'close_notes', 'u_assignee_history', 'merged', 'merged_string',
       'business_duration', 'u_reassignment_count_assigne',
       'reassignment_count', 'cleaned_string', 'isEnglish', 'user_comment',
       'agent_comment', 'problem', 'solution', 'solution_sorted'],
      dtype='object')

In [ ]:
test_text = df.loc[10, "problem"]

In [ ]:
test_text

"Please reset the CITRIX password and inform new password through user's mobile number.\n2021-01-01 13:06:07 - [name_5] (Additional comments)\nAdditional Location Information: \nAffected Device: \nIssue description: Please reset the CITRIX password and inform new password through user's mobile number.\nContact details: +91[phoneNumber_1819]011\n\n\nnan\n"

In [ ]:
preprocessed_text = []
preprocessed_text_time = []
sentences_count = []
words_count = []
words_per_sentence = []
stop_words_count = []
words_count_no_stop_words = []
words_per_sentence_no_stop_words = []
nouns_count = []
verbs_count = []
pronoun_count = []
flesch = []
coleman = []
gunning = []
open_close = []
open_now = []
sentiment_score = []
sentiment = []
language_confidence = []
inquery_intensity = []
special_characters = []
bulleted_list = []
bulleted_list_count = []
spelling_mistakes = []
verb_noun_combinations = []
pos_tagging_max_depth = []
specific_word_count = []
# unknown_words_count = []
refs = []
links_count = []
attachments_count = []
interactions_count = []



for index, row in df.iterrows():
    ticket = str(row['problem'])
    ticket_cleantxt, ticket_with_time = preprocess(ticket)
    preprocessed_text.append(ticket_cleantxt)
    preprocessed_text_time.append(ticket_with_time)
    my_ticket = Scoring(ticket_cleantxt.strip(), ticket.strip())
    ref = my_ticket.get_refrences()
    refs.append(ref)
    s1, s2 = my_ticket.get_sentences()
    sentences_count.append(s2)
    words_count.append(my_ticket.get_words())
    words_per_sentence.append(my_ticket.words_per_sentences())
    nsp1, nsp2, nsp3 = my_ticket.no_stopwords()
    stop_words_count.append(nsp1)
    words_count_no_stop_words.append(nsp2)
    words_per_sentence_no_stop_words.append(nsp3)
    nouns_count.append(my_ticket.get_nouns())
    verbs_count.append(my_ticket.get_verbs())
    pronoun_count.append(my_ticket.get_pronouns())
    r1, r2, r3 = my_ticket.get_readablity()
    flesch.append(r1)
    coleman.append(r2)
    gunning.append(r3)

    # no opened_at 
    #d1 = str(row['opened_at'])[:10].strip()
    #d2 = str(row['closed_at'])[:10].strip()
    #df1, df2 = my_ticket.get_datediff(d1, d2)
    #open_close.append(df1)
    #open_now.append(df2)

    sentiment.append(my_ticket.get_sentiment()[0])
    sentiment_score.append(my_ticket.get_sentiment()[1])
    try:
        language_confidence.append(my_ticket.get_language_confidence())
    except:
        language_confidence.append(-1)

    inquery_intensity.append(my_ticket.get_inquery_intensity())
    special_characters.append(my_ticket.get_special_characters())
    bl1, bl2 = my_ticket.get_bulleted_list()
    bulleted_list.append(bl1)
    bulleted_list_count.append(bl2)

    # try:
    #   spelling_mistakes.append(my_ticket.get_spelling_mistakes())
    # except:
    #   spelling_mistakes.append(0)
    pos_tagging_max_depth.append(my_ticket.get_max_depth())
    verb_noun_combinations.append(my_ticket.get_verb_noun_combinations()[0])
    specific_word_count.append(my_ticket.get_specific_words_count())

    # unknown_words_count.append(my_ticket.get_unknown_words_count())
    l1, l2 = Scoring().get_links(ticket)
    links_count.append(l2)
    attachments_count.append(Scoring().get_attachments(ticket))
    interactions_count.append(Scoring().get_interacts(ticket))


    if index % 1000 == 0:
        print(index)

df['clean_problem'] = preprocessed_text
df['clean_problem_with_time'] = preprocessed_text_time
df['sentences_count(Problem)'] = sentences_count
df['words_count(Problem)'] = words_count
df['words_per_sentence(Problem)'] = words_per_sentence
df['stop_words_count(Problem)'] = stop_words_count
df['words_count_no_stop_words(Problem)'] = words_count_no_stop_words
df['words_per_sentence_no_stop_words(Problem)'] = words_per_sentence_no_stop_words
df['nouns_count(Problem)'] = nouns_count
df['verbs_count(Problem)'] = verbs_count
df['pronoun_count(Problem)'] = pronoun_count
df['Flesch(Problem)'] = flesch
df['Coleman(Problem)'] = coleman
df['Gunning(Problem)'] = gunning
# df['tfidf_all(Problem)'] = ticket_tfidfall
# df['tfidf_mean(Problem)'] = ticket_tfidfmean
df['sentiment_score(Problem)'] = sentiment_score
df['sentiment(Problem)'] = sentiment
df['language_confidence(Problem)'] = language_confidence
df['inquery_intensity(Problem)'] = inquery_intensity 
df['special_characters(Problem)'] = special_characters
df['bulleted_list(Problem)'] = bulleted_list
df['spelling_mistakes(Problem)'] = spelling_mistakes
df['pos_tagging_max_depth(Problem)'] = pos_tagging_max_depth
df['verb_noun_combinations(Problem)'] = verb_noun_combinations
df['specific_word_count(Problem)'] = specific_word_count
# df['unknown_words_count(Problem)'] = unknown_words_count
df['bulleted_list_count(Problem)'] = bulleted_list_count
df['inquery_intensity(Problem)'] = inquery_intensity 
df['links_count(Problem)'] = links_count
df['attachments_count(Problem)'] = attachments_count


df.to_excel('output_problem_feutures_final.xlsx')


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000


ValueError: ignored

In [12]:
spelling_mistakes = []
unknown_words_count = []
refs = []

for index, row in df.iterrows():
    ticket = str(row['problem'])
    ticket_cleantxt = str(row['clean_problem'])
    my_ticket = Scoring(ticket_cleantxt.strip(), ticket.strip())
    ref = my_ticket.get_refrences()
    refs.append(ref)

    try:
      spelling_mistakes.append(my_ticket.get_spelling_mistakes())
    except:
      spelling_mistakes.append(0)


    unknown_words_count.append(my_ticket.get_unknown_words_count())



    if index % 1000 == 0:
        print(index)

df['refrences(Problem)'] = refs
df['spelling_mistakes(Problem)'] = spelling_mistakes
df['unknown_words_count(Problem)'] = unknown_words_count


df.to_excel('/content/drive/MyDrive/output_problem_feutures_final.xlsx')


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000


In [19]:
def preprocess_lem(text):
    alph_ticket = re.sub(r'[^a-zA-Z ]', '', text)
    lemdoc = nlp(alph_ticket)
    lemticket = ' '.join([token.lemma_ for token in lemdoc])
    return lemticket

corpus = []
for index, row in df.iterrows():
    ticket = str(row['clean_problem'])
    text = preprocess_lem(ticket)
    corpus.append(text)

ticket_tfidfall = []
ticket_tfidfmean = []
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(corpus)
feature_names = tfidf_vectorizer.get_feature_names()
for i in range(len(corpus)):
    doc = i
    count = 0
    feature_index = tfidf_vectorizer_vectors[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_vectorizer_vectors[doc, x] for x in feature_index])
    sum = 0
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
        count += 1
        sum += s
    ticket_tfidfall.append(sum)
    try:
        ticket_tfidfmean.append(sum/count)
    except:
        ticket_tfidfmean.append(0)
    if i % 1000 == 0:
       print(i)

df['tfidf_all(problem)'] = ticket_tfidfall
df['tfidf_mean(problem)'] = ticket_tfidfmean

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000


In [20]:
df.to_excel('/content/drive/MyDrive/output_problem_feutures_final2.xlsx')

In [ ]:
preprocessed_text = []
preprocessed_text_time = []
sentences_count = []
words_count = []
words_per_sentence = []
stop_words_count = []
words_count_no_stop_words = []
words_per_sentence_no_stop_words = []
nouns_count = []
verbs_count = []
pronoun_count = []
flesch = []
coleman = []
gunning = []
sentiment_score = []
sentiment = []
language_confidence = []
inquery_intensity = []
special_characters = []
bulleted_list = []
bulleted_list_count = []
verb_noun_combinations = []
pos_tagging_max_depth = []
specific_word_count = []
links_count = []
attachments_count = []
interactions_count = []



for index2, row in df.iterrows():
    ticket = str(row['solution_sorted'])
    ticket_cleantxt, ticket_with_time = preprocess(ticket)
    preprocessed_text.append(ticket_cleantxt)
    preprocessed_text_time.append(ticket_with_time)
    my_ticket = Scoring(ticket_cleantxt.strip(), ticket.strip())
    s1, s2 = my_ticket.get_sentences()
    sentences_count.append(s2)
    words_count.append(my_ticket.get_words())
    words_per_sentence.append(my_ticket.words_per_sentences())
    nsp1, nsp2, nsp3 = my_ticket.no_stopwords()
    stop_words_count.append(nsp1)
    words_count_no_stop_words.append(nsp2)
    words_per_sentence_no_stop_words.append(nsp3)
    nouns_count.append(my_ticket.get_nouns())
    verbs_count.append(my_ticket.get_verbs())
    pronoun_count.append(my_ticket.get_pronouns())
    r1, r2, r3 = my_ticket.get_readablity()
    flesch.append(r1)
    coleman.append(r2)
    gunning.append(r3)

    sentiment.append(my_ticket.get_sentiment()[0])
    sentiment_score.append(my_ticket.get_sentiment()[1])
    try:
        language_confidence.append(my_ticket.get_language_confidence())
    except:
        language_confidence.append(-1)

    inquery_intensity.append(my_ticket.get_inquery_intensity())
    special_characters.append(my_ticket.get_special_characters())
    bl1, bl2 = my_ticket.get_bulleted_list()
    bulleted_list.append(bl1)
    bulleted_list_count.append(bl2)

    pos_tagging_max_depth.append(my_ticket.get_max_depth())
    verb_noun_combinations.append(my_ticket.get_verb_noun_combinations()[0])
    specific_word_count.append(my_ticket.get_specific_words_count())

    l1, l2 = Scoring().get_links(ticket)
    links_count.append(l2)
    attachments_count.append(Scoring().get_attachments(ticket))
    interactions_count.append(Scoring().get_interacts(ticket))


    if index2 % 1000 == 0:
        print(index2)

df['clean_solution'] = preprocessed_text
df['clean_solution_with_time'] = preprocessed_text_time
df['sentences_count(solution)'] = sentences_count
df['words_count(solution)'] = words_count
df['words_per_sentence(solution)'] = words_per_sentence
df['stop_words_count(solution)'] = stop_words_count
df['words_count_no_stop_words(solution)'] = words_count_no_stop_words
df['words_per_sentence_no_stop_words(solution)'] = words_per_sentence_no_stop_words
df['nouns_count(solution)'] = nouns_count
df['verbs_count(solution)'] = verbs_count
df['pronoun_count(solution)'] = pronoun_count
df['Flesch(solution)'] = flesch
df['Coleman(solution)'] = coleman
df['Gunning(solution)'] = gunning
df['sentiment_score(solution)'] = sentiment_score
df['sentiment(solution)'] = sentiment
df['language_confidence(solution)'] = language_confidence
df['inquery_intensity(solution)'] = inquery_intensity 
df['special_characters(solution)'] = special_characters
df['bulleted_list(solution)'] = bulleted_list
df['pos_tagging_max_depth(solution)'] = pos_tagging_max_depth
df['verb_noun_combinations(solution)'] = verb_noun_combinations
df['specific_word_count(solution)'] = specific_word_count
df['bulleted_list_count(solution)'] = bulleted_list_count
df['inquery_intensity(solution)'] = inquery_intensity 
df['links_count(solution)'] = links_count
df['attachments_count(solution)'] = attachments_count


df.to_excel('/content/drive/MyDrive/output_solution_feutures_final.xlsx')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000


In [13]:
df = pd.read_excel('/content/drive/MyDrive/output_solution_feutures_final.xlsx', sheet_name=0)
print("Succesfully read excel")

Succesfully read excel


In [14]:
spelling_mistakes = []
unknown_words_count = []
refs = []

for index, row in df.iterrows():
    ticket = str(row['solution_sorted'])
    ticket_cleantxt = str(row['clean_solution'])
    my_ticket = Scoring(ticket_cleantxt.strip(), ticket.strip())
    ref = my_ticket.get_refrences()
    refs.append(ref)

    try:
      spelling_mistakes.append(my_ticket.get_spelling_mistakes())
    except:
      spelling_mistakes.append(0)


    unknown_words_count.append(my_ticket.get_unknown_words_count())



    if index % 1000 == 0:
        print(index)

df['refrences(Solution)'] = refs
df['spelling_mistakes(Solution)'] = spelling_mistakes
df['unknown_words_count(Solution)'] = unknown_words_count


df.to_excel('/content/drive/MyDrive/output_solution_feutures_final2.xlsx')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000


In [16]:
def preprocess_lem(text):
    alph_ticket = re.sub(r'[^a-zA-Z ]', '', text)
    lemdoc = nlp(alph_ticket)
    lemticket = ' '.join([token.lemma_ for token in lemdoc])
    return lemticket

corpus = []
for index, row in df.iterrows():
    ticket = str(row['clean_solution'])
    text = preprocess_lem(ticket)
    corpus.append(text)

ticket_tfidfall = []
ticket_tfidfmean = []
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(corpus)
feature_names = tfidf_vectorizer.get_feature_names()
for i in range(len(corpus)):
    doc = i
    count = 0
    feature_index = tfidf_vectorizer_vectors[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_vectorizer_vectors[doc, x] for x in feature_index])
    sum = 0
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
        count += 1
        sum += s
    ticket_tfidfall.append(sum)
    try:
        ticket_tfidfmean.append(sum/count)
    except:
        ticket_tfidfmean.append(0)
    if i % 1000 == 0:
       print(i)

df['tfidf_all(solution)'] = ticket_tfidfall
df['tfidf_mean(solution)'] = ticket_tfidfmean

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000


In [17]:
df.to_excel('/content/drive/MyDrive/output_solution_feutures_final2.xlsx')